# LIDAR Camera object

### Use get_frame() to get the depth and color frames

get_frame() returns arg1, arg2, arg3
arg1 = is valid (boolean)
arg2 = depth frame (np array)
arg3 = color frame (np array - resized to depth frame dimensions)

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2 as cv

class LIDAR_Object:
    def __init__(self):
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()
    
        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))
    
        found_rgb = False
        for s in device.sensors:
            if s.get_info(rs.camera_info.name) == 'RGB Camera':
                found_rgb = True
                break
        if not found_rgb:
            print("The demo requires Depth camera with Color sensor")
            exit(0)
    
        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        
        if device_product_line == 'L500':
            config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
        else:
            config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        # Start streaming
        self.pipeline.start(config)
        
    def get_frame(self):
        frames = self.pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv.applyColorMap(cv.convertScaleAbs(depth_image, alpha=0.03), cv.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape
        
        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            color_image = cv.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv.INTER_AREA)
        #    images = np.hstack((color_image, depth_colormap))
        #else:
        #    images = np.hstack((color_image, depth_colormap))
        
        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_image, color_image

    def release(self):
        self.pipeline.stop()

In [2]:
cam = LIDAR_Object()

In [15]:
import cv2 as cv

counter = 0
def save_frame(event, x, y, args, params):
    if event == cv.EVENT_LBUTTONDOWN:
        global counter
        global img
        for i in range(10):
            counter+=1
            print('Click')
            cv.imwrite(str(counter)+'.jpg', img)

# Create mouse event
cv.namedWindow("Color frame")
cv.setMouseCallback("Color frame", save_frame)

while True:
    ret, depth_frame, color_frame = cam.get_frame()
    img = color_frame
    cv.imshow("depth frame", depth_frame)
    cv.imshow("Color frame", color_frame)
    key = cv.waitKey(1)
    if key == 27:
        cv.destroyAllWindows()
        break
    

Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click
Click


### Mouse cursor distance

In [7]:
import cv2 as cv

point = (400, 300)

def show_distance(event, x, y, args, params):
    global point
    point = (x, y)

# Create mouse event
cv.namedWindow("Color frame")
cv.setMouseCallback("Color frame", show_distance)

while True:
    ret, depth_frame, color_frame = cam.get_frame()

    # Show distance for a specific point
    cv.circle(color_frame, point, 4, (0, 0, 255))
    distance = depth_frame[point[1], point[0]]

    cv.putText(color_frame, "{}mm".format(distance), (point[0], point[1] - 20), cv.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)

    cv.imshow("depth frame", depth_frame)
    cv.imshow("Color frame", color_frame)
    key = cv.waitKey(1)
    if key == 27:
        cv.destroyAllWindows()
        break
    

### Hand tracking

In [3]:
import cv2 as cv
import mediapipe as mp
import time

# Define currently used camera
USEDCAMERA = 0
# Define camera width and height
CAMWIDTH = 640
CAMHEIGHT = 480
# Define text color
TEXTCOLOR = (100, 150, 210)


# The hand detector model class.
# Initialize with empty parenthesis or pass the desired values in the object initialization.
# The arguments passed are the arguments needed to create the mediapipe Hands model
class HandDetector:
    def __init__(self,
                 static_image_mode=False,
                 max_num_hands=2,
                 model_complexity=1,
                 min_detection_confidence=0.5,
                 min_tracking_confidence=0.5):
        self.results = None
        self.mode = static_image_mode
        self.max_hands = max_num_hands
        self.complexity = model_complexity
        self.minimal_detection_confidence = min_detection_confidence
        self.minimal_tracking_confidence = min_tracking_confidence
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(self.mode, self.max_hands, self.complexity,
                                         self.minimal_detection_confidence,
                                         self.minimal_tracking_confidence)  # Create a mediapipe Hands class object.
        self.mp_draw_utility = mp.solutions.drawing_utils  # Create a mediapipe drawing utility to draw the points

    # find_hands() takes a frame, activates mediapipe's hand tracking algorithm and returns the frame with the hands
    # marked (points and connecting lines). As default the 'draw' argument is set to 'True'. Set to 'False' to cancel
    # drawing the points and lines on the frame.
    def find_hands(self, frame, draw=True):
        rgb_frame = cv.cvtColor(frame,  # Convert the image to RGB. mediapipe uses RGB images
                                cv.COLOR_BGR2RGB)  # and open-cv uses BGR images.
        self.results = self.hands.process(rgb_frame)  # Process the frame using the Hands.process() method
        if self.results.multi_hand_landmarks:  # If True, a hand or more were found. Else equals to None
            for hand in self.results.multi_hand_landmarks:  # Loop through the found hands and perform actions on each
                if draw:
                    self.mp_draw_utility.draw_landmarks(frame, hand,  # Draw the 21 points
                                                        self.mp_hands.HAND_CONNECTIONS)  # and the connecting lines
                    # on each hand
        return frame

    # find_position() takes a frame and the hand number to be processed (set as default to '0' - the first hand in
    # the list), finds the position of each point and returns that list
    def find_landmarks(self, frame, hand_number=USEDCAMERA):
        landmarks = []
        if self.results.multi_hand_landmarks:  # If True, a hand or more were found. Else equals to None
            hand = self.results.multi_hand_landmarks[hand_number]
            for index, landmark in enumerate(hand.landmark):  # Match the index of the landmark to the landmark
                frame_height, frame_width, channels = frame.shape
                center_x, center_y = int(landmark.x * frame_width), int(landmark.y * frame_height)
                landmarks.append([index, center_x, center_y])
        return landmarks

    def run(self, camera):
        while True:
            ret, depth_frame, color_frame = cam.get_frame()
            frame = self.find_hands(color_frame)
            self.find_landmarks(frame)
            if ret:  # If a frame exists
                cv.imshow('Camera', frame)  # Display the current frame (BGR frame)
                if cv.waitKey(1) & 0xFF == ord('q'):  # Press 'q' key to stop and exit
                    break
            else:  # No frame exists, break the loop
                break
        cv.destroyAllWindows()

In [4]:
htm = HandDetector()
cam = LIDAR_Object()

In [5]:
htm.run(cam)

### Pose/Skeleton recognition

In [6]:
import cv2 as cv
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5, enable_segmentation=True) as pose:
    while True:
        success, depth, image = cam.get_frame()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # Flip the image horizontally for a selfie-view display.
        cv.imshow('MediaPipe Pose', cv.flip(image, 1))
        if cv.waitKey(5) & 0xFF == 27:
            cv.destroyAllWindows()
            break


# Save images / Dataset prep

In [ ]:
cam = LIDAR_Object()

In [ ]:
import cv2 as cv

counter = 0
def save_frame(event, x, y, args, params):
    if event == cv.EVENT_LBUTTONDOWN:
        global counter
        global img
        counter+=1
        cv.imwrite(str(counter)+'.jpg', img)

# Create mouse event
cv.namedWindow("Color frame")
cv.setMouseCallback("Color frame", save_frame)

while True:
    ret, depth_frame, color_frame = cam.get_frame()
    img = color_frame
    cv.imshow("depth frame", depth_frame)
    cv.imshow("Color frame", color_frame)
    key = cv.waitKey(1)
    if key == 27:
        cv.destroyAllWindows()
        break
    